### 限制 token 數量並自動刪除過多的訊息

In [ ]:
!pip install tiktoken -q

In [ ]:
# 匯入 tiktoken 模組，用於處理文字編碼，將訊息轉換為 token
import tiktoken

# 定義函數 num_tokens_from_messages，用於計算訊息列表所使用的 token 數量
# 來源：https://platform.openai.com/docs/guides/gpt/managing-tokens
def num_tokens_from_messages(messages, model="gpt-4o-mini"):
  """返回一組訊息中使用的 token 數量。"""
  
  try:
      # 根據指定的模型取得編碼方式
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      # 如果指定的模型不存在，則使用基礎編碼方式 "cl100k_base"
      encoding = tiktoken.get_encoding("cl100k_base")
      
  # 如果指定的模型為 "gpt-4o-mini"，計算該模型的 token 使用情況
  if model == "gpt-4o-mini":  # 注意：未來的模型可能會有不同的計算方式
      num_tokens = 0  # 初始化 token 計數器
      
      # 遍歷每條訊息，計算每條訊息的 token 數量
      for message in messages:
          num_tokens += 4  # 每條訊息都會有 4 個額外 token 表示訊息格式 <im_start>{role/name}\n{content}<im_end>\n
          
          # 計算訊息中每個欄位（如 role, content, name 等）的 token 數量
          for key, value in message.items():
              # 使用 encoding.encode 將訊息的值轉換為 token 數量並累加
              num_tokens += len(encoding.encode(value))
              
              # 如果有 "name" 欄位，則角色 "role" 可以省略，減去 1 個 token
              if key == "name":
                  num_tokens += -1  # "role" 始終需要並且僅佔用 1 個 token
      
      # 每次回應都會包含 2 個額外的 token 用於 "<im_start>assistant"
      num_tokens += 2  
      return num_tokens  # 回傳計算出的總 token 數量
  
  # 若模型並非 "gpt-4o-mini"，則拋出未實作的錯誤提示
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() 函數目前未實作模型 {model} 的 token 計算方法。
  參見 https://github.com/openai/openai-python/blob/main/chatml.md 以了解訊息如何轉換為 token。""")


In [ ]:
# 定義 handle_truncate 函數，用於在訊息列表的 token 總量超過指定上限時，移除部分訊息
def handle_truncate(messages, max_tokens=4096):
  
  # 當訊息的 token 總數大於指定的 max_tokens 且訊息數量多於 1 時，不斷執行截斷操作
  while num_tokens_from_messages(messages) > max_tokens and len(messages) > 1:
    
    # 遍歷訊息列表中的每條訊息
    for index, message in enumerate(messages):
        # 如果訊息的角色不是 "system"，則可以刪除該訊息
        if message['role'] != 'system':
          print(f"remove: {message}")  # 印出即將刪除的訊息內容
          messages.pop(index)  # 從列表中移除這條訊息
          break  # 刪除一條訊息後，跳出 for 循環，重新計算 token 總數
  
  # 當 token 總數不再超過上限，印出剩餘的訊息列表和 token 總數
  print("------ 剩餘的 messages ------")
  print("token 量: ", num_tokens_from_messages(messages))
  
  # 返回截斷後的訊息列表
  return messages

測試的目的：此測試模擬了一組訊息在超過 token 限制時的截斷過程，確保 handle_truncate 函數正確地移除最早的 user 和 assistant 訊息，並在滿足 token 限制後返回剩餘訊息列表。這樣的處理方式有助於保留對話的上下文，並確保符合 API token 上限。

In [ ]:
# 當 messages 超過閥值時，把最前面的 user & assistant 對話砍了
messages = [
    { "role": "system", "content": "You're a helpful assistant"},
    { "role": "user", "content": "你好，今天新竹天氣如何?" },
    { "role": "assistant", "content": "今天新竹早上出太陽，下午下雨" },
    { "role": "user", "content": "我正在嘗試了解有監督學習和無監督學習的區別。你可以解釋一下嗎？" },
    { "role": "assistant", "content": "當然可以！有監督學習涉及在有標籤的數據集上訓練模型，這意味著數據集中的每個範例都與正確答案配對。模型然後從這些範例中學習。另一方面，無監督學習處理未標籤的數據。目標是在數據中尋找模式或關係，而不需要明確被告知要尋找什麼。" },
    { "role": "user", "content": "我明白了。所以，在有監督學習中，我們始終需要有標籤的數據嗎？" },
    { "role": "assistant", "content": "是的，沒錯。在有監督學習中，擁有標籤的數據是必要的，因為它為模型提供了輸入和期望的輸出，使模型可以學習它們之間的關係。" },
    { "role": "user", "content": "那對於無監督學習，有沒有常見的算法或方法？" },
    { "role": "assistant", "content": "當然有！一些常見的無監督學習方法包括聚類（如 K-means）和降維技術（如 PCA 或 t-SNE）。這些方法的目的是基於數據中的固有結構或模式來分組數據點或減少特徵的數量。" },
    { "role": "user", "content": "明白了，謝謝你的解釋！" },
    { "role": "assistant", "content": "不客氣！如果你還有其他問題，隨時告訴我。祝你學習愉快！" }
]

handle_truncate(messages, max_tokens = 500)